# 演習5

入札者の数を $n$ 、財の個数を $m$ とする。

メカニズム1において、入札額が $m$ 番目に多い買い手まで財を割り当てる。

今、財の個数は３個かつ、入札額 $\mathbf{b}$ は下記の様に与えられている。

$$
\mathbf{b} =
\begin{pmatrix}
1 \\
5 \\
3 \\
2 \\
7 \\
5
\end{pmatrix}
$$

よって、入札額が高い上位３人の入札者に財が割り当てらるので割り当てを表すベクトル $\mathbf{x}$ は下記のようになる。

$$
\mathbf{x} =
\begin{pmatrix}
0 \\
1 \\
0 \\
0 \\
1 \\
1
\end{pmatrix}
$$


また、メカニズム１では入札額と支払額は等しくなるので、支払額を表すベクトル $\mathbf{p}$  は下記の様になる。

$$
\mathbf{p} =
\begin{pmatrix}
0 \\
5 \\
0 \\
0 \\
7 \\
5
\end{pmatrix}
$$



# 演習6

入札者の数を $n$ 、財の個数を $m$ とする。

次点価格オークションメカニズムにおいても、入札額が $m$ 番目に多い買い手まで財を割り当てる。

今、財の個数は３で、入札額は下記の $\mathbf{b}$ の様に与えられている。

$$
\mathbf{b} =
\begin{pmatrix}
1 \\
5 \\
3 \\
2 \\
7 \\
5
\end{pmatrix}
$$

よって、入札額が高い上位３人の入札者に財が割り当てらるので割り当てを表すベクトル $\mathbf{x}$ は下記のようになる。

$$
\mathbf{x} =
\begin{pmatrix}
0 \\
1 \\
0 \\
0 \\
1 \\
1
\end{pmatrix}
$$


また、次点価格オークションメカニズムでは、支払額は $m+1$ 番目に高い入札額となる。今回の場合、$4$ 番目に高い入札額は $5$ である。よって、支払額を表すベクトル $\mathbf{p}$  は下記の様になる。

$$
\mathbf{p} =
\begin{pmatrix}
0 \\
5 \\
0 \\
0 \\
5 \\
5
\end{pmatrix}
$$

# 演習7

# 演習8



# 演習 9

演習 5 で売り手が出品数を 2 に変更した場合,次点価格オークションメカニ
ズムでの売り手の収入を計算せよ.このとき,売り手の収入はどうなるだろう
か?売り手は出品する財の個数を調整することで収入を増やせるだろうか?

# 演習10

# 演習11

複数需要オークションで割当と,支払い価格を計算する VCG メカニズムを
動的計画法を用いて実装せよ。

## 初期条件

- 誰もいないときは価値ゼロ：

  $$
  V[0, \ell] = 0 \quad (\ell = 0, 1, \ldots, m)
  $$

- 財が 0 個のときも価値ゼロ：

  $$
  V[k, 0] = 0 \quad (k = 0, 1, \ldots, n)
  $$


In [ ]:
from typing import List

class cal_dp:
    def __init__(self, n: int, m: int, v: List[int]):
        self.n = n               # 入札者数
        self.m = m               # 財の総数
        self.v = v               # v[i]: 入札者iの入札額
        self.dp = [[0 for _ in range(m + 1)] for _ in range(n + 1)]
        # allocation[i] = units given to bidder i (1-based like DP)
        self.allocation = [0 for _ in range(n + 1)] # self.allocation[i]: i番目の入札者に割り当てられた財の数
        self.dp = self.resolve_dp(self.v) # DPテーブルを埋める
        self.allocation = self.cal_assignment(self.v, self.dp)[1:] # 割り当て計算
        self.payment = self.cal_payment() # 支払い計算（必要に応じて実装）
        # print results
        print("payment:", self.payment)
        print("Allocation:", self.allocation)  
        print("Max value:", self.dp[self.n][self.m])
        print("DP Table:")
        for row in self.dp:
            print(row)
    def fill_with_zero(self,values:List[int],idx:int):
        values[idx] = 0
        return values

    def resolve_dp(self,values:List[int]=None):
        """
        DPテーブルを埋める関数
        引数:
            values: 入札額リスト
        帰り値:
            tmp:埋められたDPテーブル
        """
        tmp:List[List[int]] = [[0 for _ in range(self.m + 1)] for _ in range(self.n + 1)]
        for i in range(1, self.n + 1):
            for j in range(0, self.m + 1):
                max_val = -1
                # try giving z units to bidder i
                for z in range(0, j + 1):
                    cand = self.dp[i - 1][j - z] + values[i - 1] * z
                    if cand > max_val:
                        max_val = cand
                tmp[i][j] = max_val
        return tmp

    def cal_assignment(self,values:List[int],dp:List[List[int]]):
        """
        割り当て計算を行う関数
        引数:
            values: 入札額リスト
            dp: DPテーブル
        帰り値:
            allocations: 各入札者に割り当てられた財の数
        """
        allocations = [0 for _ in range(self.n + 1)]
        m_tmp = self.m
        for i in range(self.n, 0, -1):
            max_val = -1
            allocation = 0
            for z in range(0, m_tmp + 1):
                if max_val < dp[i - 1][self.m - z] + values[i - 1] * z:
                    max_val = dp[i - 1][self.m - z] + values[i - 1] * z
                    allocation = z

            allocations[i] = allocation
            m_tmp -= allocation
        return allocations
    def cal_payment(self):
        payments:List[int] = []
        i_ignored_allocations:List[List[int]] = []
        self.ignore_i: List[List[List[int]]] = ([])  # i番目の入札者を無視した場合のDPテーブル
        for i in range(self.n):
            tmp_v = self.v.copy()
            tmp_v = self.fill_with_zero(tmp_v, i)
            i_ignored_allocations.append(
                self.cal_assignment(tmp_v, self.resolve_dp(tmp_v))[1:]
            )
        for j in range(self.n):
            add_value = 0
            minus_value = 0
            for i in range(self.n):
                if i != j:
                    add_value += (
                        i_ignored_allocations[j][i]
                        * self.v[i]
                    )
                    minus_value += (
                        self.allocation[i]
                        * self.v[i]
                    )
            payments.append(add_value - minus_value)
        return payments
# 動作確認
cal_dp(3, 4, [2, 3, 4])

payment: [0, 0, 12]
Allocation: [0, 0, 4]
Payments:
Max value: 16
DP Table:
[0, 0, 0, 0, 0]
[0, 2, 4, 6, 8]
[0, 3, 6, 9, 12]
[0, 4, 8, 12, 16]


# 演習12

In [10]:
import random
num_bidders = 50
num_items = 30
values = [random.randint(10, 100) for _ in range(num_bidders)]
print(f"入札者数: {num_bidders}, 財の個数: {num_items}, 入札額: {values}")

入札者数: 50, 財の個数: 30, 入札額: [32, 24, 97, 37, 29, 22, 26, 17, 85, 29, 31, 88, 42, 77, 75, 21, 17, 85, 83, 66, 33, 57, 88, 32, 22, 18, 45, 57, 48, 28, 77, 55, 89, 98, 55, 25, 32, 26, 14, 70, 52, 52, 42, 89, 18, 48, 81, 91, 76, 88]


# 演習13

In [ ]:
from typing import List
class MultiUnitVCGAuctionSolver:

    def __init__(self, n: int, m: int,e:List[List[int]], D:List[List[List[int]]]):
        self.n = n               # 入札者数
        self.m = m               # 財の個数
        self.D = D               # [D[i][j][0], D[i][j][1]]: 入札者iがj番目の需要区間
        self.e = e               # e[i][j]: 入札者iのj番目の需要区間の傾き
        self.T = [[0 for _ in range(m + 1)] for _ in range(n + 1)] # DPテーブル
        self.solve()
    def v_i_x(self,i:int,x:int):
        """
        入札者iがx個の財を受け取ったときの入札額を返す
        """
        e_i = self.e[i]
        for j in range(len(e_i)):
            if self.D[i][j][0] <= x <= self.D[i][j][1]:
                return e_i[j] * x
        return 0
    def solve(self):
        """
        DPテーブルを埋める
        """
        for i in range(1, self.n + 1):
            for h in range(0, self.m + 1):
                max_val = self.T[i-1][h]
                for z in range(1, h + 1):
                    cand = self.T[i - 1][h - z] + self.v_i_x(i-1,z)
                    if cand > max_val:
                        max_val = cand
                self.T[i][h] = max_val
        print("Total Value Table T:")
        for row in self.T:
            print(row)
                   
            
        
D = [[[1,4],[5,6],[7,9]],
     [[2,3],[4,8],[9,10]],
     [[1,2],[3,5],[6,7]]]
e = [[3,1,1],[5,4,2],[7,6,4]]
MultiUnitVCGAuctionSolver(3,6,e,D)

max_value = -1
best_allocation = (0, 0, 0)
for one in range(7):
    for two in range(7-one):
        value = 0
        three = 6 - one - two
        for i in range(3):
            if D[0][i][0] <= one <= D[0][i][1]:
                value += e[0][i] * one
            if D[1][i][0] <= two <= D[1][i][1]:
                value += e[1][i] * two
            if D[2][i][0] <= three <= D[2][i][1]:
                value += e[2][i] * three
        if value > max_value:
            max_value = value
            best_allocation = (one, two, three)
print(f"Max value: {max_value}, Best allocation: {best_allocation}")

Total Value Table T:
[0, 0, 0, 0, 0, 0, 0]
[0, 3, 6, 9, 12, 12, 12]
[0, 3, 10, 15, 18, 21, 24]
[0, 7, 14, 18, 24, 30, 34]
Max value: 34, Best allocation: (0, 2, 4)


In [16]:
import random


def generate_problem():
    # --- 各パラメータ ---
    bidders = random.choice([10, 50, 100])  # 入札者数
    items = random.choice([10, 20, 50])  # 財の個数
    num_intervals = random.randint(5, 10)  # 需要閉区間の個数（一様乱数）
    slope = random.uniform(10, 100)  # 評価関数の傾き（一様乱数）

    m = 10  # 区間生成の上限（問題設定より）

    # --- 需要閉区間を生成 ---
    # 区間端点を [0, m] の範囲で num_intervals + 1 個ランダムに生成し、昇順にソート
    points = sorted(random.sample(range(0, m + 1), num_intervals + 1))

    intervals = []
    for i in range(num_intervals):
        start = points[i]
        end = points[i + 1]
        # start == end の場合を除外
        if start < end:
            intervals.append([start, end])

    # --- 出力 ---
    problem = {
        "入札者数": bidders,
        "財の個数": items,
        "需要閉区間の個数": num_intervals,
        "需要閉区間": intervals,
        "評価関数の傾き": round(slope, 2),
    }
    return problem


# 例として1つ生成
problem = generate_problem()
for k, v in problem.items():
    print(f"{k}: {v}")

入札者数: 10
財の個数: 50
需要閉区間の個数: 10
需要閉区間: [[0, 1], [1, 2], [2, 3], [3, 4], [4, 5], [5, 6], [6, 7], [7, 8], [8, 9], [9, 10]]
評価関数の傾き: 26.95


In [77]:
def make_problem(n:int,m:int):
    """
    n: 入札者数 
    m: 財の個数
    """
    D = []
    e = []
    num_demand_intervals = random.randint(5, 10)
    width = (int)(m / num_demand_intervals) - 1 
    num_intervals = 0
    tmp = 0
    reminder = m - (width + 1) * num_demand_intervals
    print("入札者数:",n)
    print("財の個数:",m)
    for i in range(n):
        D_i = []
        e_i = []
        num_intervals = 0
        tmp = 0
        reminder = m - (width + 1) * num_demand_intervals   
        while True:
            num_intervals += 1
            if num_intervals == num_demand_intervals:
                end = m
                D_i.append([tmp,end])
                e_i.append(random.randint(10,100))
                break
            else:
                start = tmp
                end = start + width
                rad = random.randint(0,reminder)
                end += rad
                reminder -= rad
                D_i.append([start,end])
                e_i.append(random.randint(10,100))
                tmp = end + 1
        D.append(D_i)
        e.append(e_i)
    print("傾き:")
    print(e)
    print("需要区間:")
    print("[")
    for row in D:
        print(row)
    print("]")

make_problem(10,20)

入札者数: 10
財の個数: 20
傾き:
[[83, 76, 48, 75, 54], [44, 48, 89, 86, 31], [53, 92, 71, 69, 85], [29, 44, 65, 38, 13], [13, 68, 77, 91, 58], [64, 51, 45, 43, 26], [42, 17, 17, 71, 22], [77, 72, 25, 73, 65], [49, 27, 90, 99, 73], [91, 19, 19, 56, 76]]
需要区間:
[
[[0, 3], [4, 7], [8, 11], [12, 15], [16, 20]]
[[0, 3], [4, 7], [8, 11], [12, 15], [16, 20]]
[[0, 3], [4, 7], [8, 11], [12, 15], [16, 20]]
[[0, 3], [4, 7], [8, 11], [12, 15], [16, 20]]
[[0, 3], [4, 7], [8, 11], [12, 15], [16, 20]]
[[0, 3], [4, 7], [8, 11], [12, 15], [16, 20]]
[[0, 3], [4, 7], [8, 11], [12, 15], [16, 20]]
[[0, 3], [4, 7], [8, 11], [12, 15], [16, 20]]
[[0, 3], [4, 7], [8, 11], [12, 15], [16, 20]]
[[0, 3], [4, 7], [8, 11], [12, 15], [16, 20]]
]


# 演習14

# 演習15

クラスHngarianとして実装する

```Python
class Hungarian:
    def __init__(self,X):

```

## 入力X


$$
\mathbf{X}
=
\begin{bmatrix}
x_{1,1} & x_{1,2} & \cdots & x_{1,N} \\
x_{2,1} & x_{2,2} & \cdots & x_{2,N} \\
\vdots  & \vdots  & \ddots & \vdots  \\
x_{N,1} & x_{N,2} & \cdots & x_{N,N}
\end{bmatrix},
\qquad 1 \le i \le N,\; 1 \le j \le N
$$


## step1

## step2

クラスHangarian内にてdef _step(self)として定義する。

## step3

クラスHangarian内にてdef _step3(self)として定義する。

## step4


In [3]:
import numpy as np
from typing import List


class Hungarian:
    N: int  # the dimension of the profit matrix
    tmp_tensor: np.ndarray

    def __init__(self, profit_matrix):
        self.profit_matrix = profit_matrix
        self.cost_matrix = self.create_cost_matrix(profit_matrix)
        self.step1()
        self.ans_pos = self.step_2_3()
        self.ans, self.ans_mat = self.ans_calculation(profit_matrix, self.ans_pos)

    def create_cost_matrix(self, profit_matrix):
        self.N = profit_matrix.shape[0]
        max_value = np.max(profit_matrix)
        return max_value - profit_matrix

    def min_zero_row(self, zero_mat, mark_zero):
        """
        The function can be splitted into two steps:
        #1 The function is used to find the row which containing the fewest 0.
        #2 Select the zero number on the row, and then marked the element corresponding row and column as False
        """

        # Find the row
        min_row = [99999, -1]

        for row_num in range(zero_mat.shape[0]):
            if np.sum(zero_mat[row_num] == True) > 0 and min_row[0] > np.sum(
                zero_mat[row_num] == True
            ):
                min_row = [np.sum(zero_mat[row_num] == True), row_num]

        # Marked the specific row and column as False
        zero_index = np.where(zero_mat[min_row[1]] == True)[0][0]
        mark_zero.append((min_row[1], zero_index))
        zero_mat[min_row[1], :] = False
        zero_mat[:, zero_index] = False

    def mark_matrix(self, mat):
        """
        Finding the returning possible solutions for LAP problem.
        """

        # Transform the matrix to boolean matrix(0 = True, others = False)
        cur_mat = mat
        zero_bool_mat = cur_mat == 0
        zero_bool_mat_copy = zero_bool_mat.copy()

        # Recording possible answer positions by marked_zero
        marked_zero = []
        while True in zero_bool_mat_copy:
            self.min_zero_row(zero_bool_mat_copy, marked_zero)

        # Recording the row and column positions seperately.
        marked_zero_row = []
        marked_zero_col = []
        for i in range(len(marked_zero)):
            marked_zero_row.append(marked_zero[i][0])
            marked_zero_col.append(marked_zero[i][1])

        # Step 2-2-1
        non_marked_row = list(set(range(cur_mat.shape[0])) - set(marked_zero_row))

        marked_cols = []
        check_switch = True
        while check_switch:
            check_switch = False
            for i in range(len(non_marked_row)):
                row_array = zero_bool_mat[non_marked_row[i], :]
                for j in range(row_array.shape[0]):
                    # Step 2-2-2
                    if row_array[j] == True and j not in marked_cols:
                        # Step 2-2-3
                        marked_cols.append(j)
                        check_switch = True

            for row_num, col_num in marked_zero:
                # Step 2-2-4
                if row_num not in non_marked_row and col_num in marked_cols:
                    # Step 2-2-5
                    non_marked_row.append(row_num)
                    check_switch = True
        # Step 2-2-6
        marked_rows = list(set(range(mat.shape[0])) - set(non_marked_row))

        return (marked_zero, marked_rows, marked_cols)

    def step1(self):
        self.tmp_tensor = self.cost_matrix.copy()
        for i in range(self.N):
            min_value = min(self.tmp_tensor[i])
            for j in range(self.N):
                self.tmp_tensor[i][j] -= min_value
        for j in range(self.N):
            col_values = [self.tmp_tensor[i][j] for i in range(self.N)]
            min_value = min(col_values)
            for i in range(self.N):
                self.tmp_tensor[i][j] -= min_value

    def step_2_3(self):
        tmp_tensor = self.tmp_tensor
        num_zero: int = 0
        while num_zero < self.N:
            pos, marked_rows, marked_cols = self.mark_matrix(np.array(tmp_tensor))
            num_zero = len(marked_rows) + len(marked_cols)
            if num_zero < self.N:
                tmp_tensor = self.adjust_matrix(tmp_tensor, marked_rows, marked_cols)
        return pos

    def adjust_matrix(self, mat, cover_rows, cover_cols):
        cur_mat = mat
        non_zero_element = []

        # Step 4-1
        for row in range(len(cur_mat)):
            if row not in cover_rows:
                for i in range(len(cur_mat[row])):
                    if i not in cover_cols:
                        non_zero_element.append(cur_mat[row][i])
        min_num = min(non_zero_element)

        # Step 4-2
        for row in range(len(cur_mat)):
            if row not in cover_rows:
                for i in range(len(cur_mat[row])):
                    if i not in cover_cols:
                        cur_mat[row, i] = cur_mat[row, i] - min_num
        # Step 4-3
        for row in range(len(cover_rows)):
            for col in range(len(cover_cols)):
                cur_mat[cover_rows[row], cover_cols[col]] = (
                    cur_mat[cover_rows[row], cover_cols[col]] + min_num
                )
        return cur_mat

    def ans_calculation(self, mat, pos):
        total = 0
        ans_mat = np.zeros((mat.shape[0], mat.shape[1]))
        for i in range(len(pos)):
            total += mat[pos[i][0], pos[i][1]]
            ans_mat[pos[i][0], pos[i][1]] = mat[pos[i][0], pos[i][1]]
        return total, ans_mat

    def get_answer(self):
        return self.ans, self.ans_mat


def main():

    profit_matrix = np.array([[7, 8, 5, 6], [6, 5, 9, 10], [4, 1, 10, 7], [3, 4, 6, 5]])
    solver = Hungarian(profit_matrix.copy())
    ans, ans_mat = solver.get_answer()
    print(f"Linear Assignment problem result: {ans:.0f}\n{ans_mat}")


if __name__ == "__main__":
    main()

Linear Assignment problem result: 31
[[ 7.  0.  0.  0.]
 [ 0.  0.  0. 10.]
 [ 0.  0. 10.  0.]
 [ 0.  4.  0.  0.]]


# 演習16

ハンガリー法を用いて $(AP)$、$(AP)^{-i}$ を計算する


In [1]:
import numpy as np
from typing import List


class Hungarian:
    N: int  # the dimension of the profit matrix
    tmp_tensor: np.ndarray

    def __init__(self, profit_matrix):
        self.profit_matrix = profit_matrix
        self.cost_matrix = self.create_cost_matrix(profit_matrix)
        self.step1()
        self.ans_pos = self.step_2_3()
        self.ans, self.ans_mat = self.ans_calculation(profit_matrix, self.ans_pos)

    def create_cost_matrix(self, profit_matrix):
        self.N = profit_matrix.shape[0]
        max_value = np.max(profit_matrix)
        return max_value - profit_matrix

    def min_zero_row(self, zero_mat, mark_zero):
        """
        The function can be splitted into two steps:
        #1 The function is used to find the row which containing the fewest 0.
        #2 Select the zero number on the row, and then marked the element corresponding row and column as False
        """

        # Find the row
        min_row = [99999, -1]

        for row_num in range(zero_mat.shape[0]):
            if np.sum(zero_mat[row_num] == True) > 0 and min_row[0] > np.sum(
                zero_mat[row_num] == True
            ):
                min_row = [np.sum(zero_mat[row_num] == True), row_num]

        # Marked the specific row and column as False
        zero_index = np.where(zero_mat[min_row[1]] == True)[0][0]
        mark_zero.append((min_row[1], zero_index))
        zero_mat[min_row[1], :] = False
        zero_mat[:, zero_index] = False

    def mark_matrix(self, mat):
        """
        Finding the returning possible solutions for LAP problem.
        """

        # Transform the matrix to boolean matrix(0 = True, others = False)
        cur_mat = mat
        zero_bool_mat = cur_mat == 0
        zero_bool_mat_copy = zero_bool_mat.copy()

        # Recording possible answer positions by marked_zero
        marked_zero = []
        while True in zero_bool_mat_copy:
            self.min_zero_row(zero_bool_mat_copy, marked_zero)

        # Recording the row and column positions seperately.
        marked_zero_row = []
        marked_zero_col = []
        for i in range(len(marked_zero)):
            marked_zero_row.append(marked_zero[i][0])
            marked_zero_col.append(marked_zero[i][1])

        # Step 2-2-1
        non_marked_row = list(set(range(cur_mat.shape[0])) - set(marked_zero_row))

        marked_cols = []
        check_switch = True
        while check_switch:
            check_switch = False
            for i in range(len(non_marked_row)):
                row_array = zero_bool_mat[non_marked_row[i], :]
                for j in range(row_array.shape[0]):
                    # Step 2-2-2
                    if row_array[j] == True and j not in marked_cols:
                        # Step 2-2-3
                        marked_cols.append(j)
                        check_switch = True

            for row_num, col_num in marked_zero:
                # Step 2-2-4
                if row_num not in non_marked_row and col_num in marked_cols:
                    # Step 2-2-5
                    non_marked_row.append(row_num)
                    check_switch = True
        # Step 2-2-6
        marked_rows = list(set(range(mat.shape[0])) - set(non_marked_row))

        return (marked_zero, marked_rows, marked_cols)

    def step1(self):
        self.tmp_tensor = self.cost_matrix.copy()
        for i in range(self.N):
            min_value = min(self.tmp_tensor[i])
            for j in range(self.N):
                self.tmp_tensor[i][j] -= min_value
        for j in range(self.N):
            col_values = [self.tmp_tensor[i][j] for i in range(self.N)]
            min_value = min(col_values)
            for i in range(self.N):
                self.tmp_tensor[i][j] -= min_value

    def step_2_3(self):
        tmp_tensor = self.tmp_tensor
        num_zero: int = 0
        while num_zero < self.N:
            pos, marked_rows, marked_cols = self.mark_matrix(np.array(tmp_tensor))
            num_zero = len(marked_rows) + len(marked_cols)
            if num_zero < self.N:
                tmp_tensor = self.adjust_matrix(tmp_tensor, marked_rows, marked_cols)
        return pos

    def adjust_matrix(self, mat, cover_rows, cover_cols):
        cur_mat = mat
        non_zero_element = []

        # Step 4-1
        for row in range(len(cur_mat)):
            if row not in cover_rows:
                for i in range(len(cur_mat[row])):
                    if i not in cover_cols:
                        non_zero_element.append(cur_mat[row][i])
        min_num = min(non_zero_element)

        # Step 4-2
        for row in range(len(cur_mat)):
            if row not in cover_rows:
                for i in range(len(cur_mat[row])):
                    if i not in cover_cols:
                        cur_mat[row, i] = cur_mat[row, i] - min_num
        # Step 4-3
        for row in range(len(cover_rows)):
            for col in range(len(cover_cols)):
                cur_mat[cover_rows[row], cover_cols[col]] = (
                    cur_mat[cover_rows[row], cover_cols[col]] + min_num
                )
        return cur_mat

    def ans_calculation(self, mat, pos):
        total = 0
        ans_mat = np.zeros((mat.shape[0], mat.shape[1]))
        for i in range(len(pos)):
            total += mat[pos[i][0], pos[i][1]]
            ans_mat[pos[i][0], pos[i][1]] = mat[pos[i][0], pos[i][1]]
        return total, ans_mat

    def get_answer(self):
        return self.ans, self.ans_mat

In [27]:
from typing import List
import sys
from typing import List


class HungarianVCGAllocator:
    n_bidders: int
    n_items: int
    values: List[List[int]]
    ans_allocation: List[int] = []
    ans_payment: List[int] = []
    x_star: List[List[int]] = []
    x_minus: List[List[List[int]]] = []
    AP: List[List[int]] = []

    def __init__(self, n_bidders: int, n_items: int, values: List[List[int]]):
        self.n_cols = len(values[0])
        self.n_rows = len(values)
        if self.n_cols != n_items or self.n_rows != n_bidders:
            raise ValueError(
                "Dimension of values does not match n_bidders and n_items."
            )
        self.n_bidders = n_bidders
        self.n_items = n_items
        self.values = values
        print("Input values:")
        for row in self.values:
            print(row)
        print("=" * 30)
        _, self.AP = Hungarian(np.array(self.values)).get_answer()
        self.x_star = [
            [1 if self.AP[i][j] != 0 else 0 for j in range(len(self.AP[i]))]
            for i in range(len(self.AP))
        ]
        for i in range(self.n_bidders):
            _, ans_mat = Hungarian(np.array(self.avoid_i(i))).get_answer()
            self.x_minus.append(
                [
                    [1 if ans_mat[k][j] != 0 else 0 for j in range(len(ans_mat[k]))]
                    for k in range(len(ans_mat))
                ]
            )
        
        self.ans_payment = self.cal_payment()
        print("Payments:", self.ans_payment)

    def cal_payment(self) -> List[int]:
        print("Calculating payments...")
        print("=" * 30)
        print("Input values:")
        for row in self.values:
            print(row)
        print("=" * 30)
        print(f"x_star:\n")
        for row in self.x_star:
            print(row)
        print("=" * 30)
        print(f"x_minus:\n")
        for i in range(self.n_bidders):
            print(f"Excluding bidder {i}:\n")
            for row in self.x_minus[i]:
                print(row)
            print("-" * 20)
        payments = []
        for i in range(self.n_bidders):
            value_without_i = 0
            value_with_i = 0
            for k in range(self.n_bidders):
                for j in range(self.n_items):
                    if k != i:
                        value_without_i += self.values[k][j] * self.x_minus[i][k][j]
                        value_with_i += self.values[k][j] * self.x_star[k][j]
            payments.append(value_without_i - value_with_i)
        return payments

    def avoid_i(self, i: int):
        arr = np.array(self.values, dtype=int).copy()
        arr[i, :] = 0
        return arr


# ---- 出力をファイルに保存 ----
sample_n_bidders = 4
sample_n_items = 4
sample_tensor = [[7, 8, 5, 6], [6, 5, 9, 10], [4, 1, 10, 7], [3, 4, 6, 5]]

import sys
from IPython.utils import io

with open("vcg_output.txt", "w", encoding="utf-8") as f:
    with io.capture_output() as captured:
        sys.stdout = f
        test_allocator = HungarianVCGAllocator(
            sample_n_bidders, sample_n_items, sample_tensor
        )
        sys.stdout = sys.__stdout__

print("✅ 出力を 'vcg_output.txt' に保存しました。")


In [29]:
import random
sys.stdout = sys.__stdout__  # 出力先を元に戻す
x = random.randint(10, 50)
print(x)

入札者:10
財の種類:14
評価額:

入札者:10
財の種類:27
評価額:

財の種類:40 

財の種類:33 

財の種類:28 

財の種類:42 

財の種類:46

財の種類:48 

22 

48
